# Multi-Layer Perceptron Neural Network

In this assignment, We will construct and train a multi-layer perceptron (MLP) model to categorize pictures into 10 classes.
## Background

![mlp](./mlp.png)

An Multi-Layer Perceptron (MLP) is a classic type of feedforward artificial neural network (ANN). 
It serves as the foundation for many more complex deep learning architectures.
An MLP is structured in a series of layers:
- Input Layer: This is where the network receives your raw data or features.
- Hidden Layer(s): One or more layers of neurons (or nodes) positioned between the input and output. 
This is where the "learning" and complex computation primarily occur. 
A network with multiple hidden layers is what makes it "deep."
- Output Layer: This final layer produces the network's prediction, such as a class probability or a continuous value.

In an MLP, each neuron in one layer is typically fully connected to every neuron in the next. 
The power of the MLP comes from its non-linear activation functions (like ReLU or sigmoid) applied within the hidden neurons. 
These functions allow the network to learn and model complex, non-linear patterns in data. 

MLP itself has a wide an a variety of applications (e.g. image classification, fraud detection, spam filtering, etc..).
It also serves as key components in today's larger scale AI models such as in Convolutional Neural Network (CNN), Deep Reinforcement Learning (DRL), Large Language Model (LLM), etc..
  
## Objectives:
- Review data preprocessing techniques (rescale, flatten, one-hot encoding, etc.).
- Practice MLP construction with arbitrary hidden layer dimensions.
- Realize a variety of activation functions (`sigmoid`, `tanh`, `relu`, `leaky_relu`).
- Realize the special softmax activation which formats features into categorical probability distributions.
- Ultilize cross entropy function to evaluate a general classification model.  
- Apply back propagation to a multi-layer gradients computation.  
- Reinforce the of model training process.

## Exercises:
<font color=582c83>

1. (5%) Parameters Initialization
2. (5%) Activation Functions
3. (10%) MLP Forward Pass Construction
4. (10%) Cross Entropy Loss and Classification Accuracy
5. (4%) Derivatives of Activation Functions
6. (18%) Gradients of Loss Back-Propagation
7. (33%) Stochastic Gradient Descent (SGD) Optimization
8. (15%) Model Analyses

</font>


## 1. Prepare Data
The CIFAR-10 dataset (Canadian Institute For Advanced Research) is a collection of images that are commonly used to train machine learning and computer vision algorithms. 
It consists of 60000 32x32 color images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.
Torchvision provides a descent pool of datasets and [CIFAR10](https://docs.pytorch.org/vision/main/generated/torchvision.datasets.CIFAR10.html) is one of them.
Wee will investigate a multi-class classification problem using this dataset.


In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms.v2 import Compose, ToImage, ToDtype
import numpy as np
import matplotlib.pyplot as plt

# Construct pre-processing pipeline
feature_transform_pipeline = Compose([
    ToImage(),
    ToDtype(torch.float32, scale=True),
    torch.flatten,
    torch.Tensor.numpy,
])
label_transform_pipeline = Compose([
    lambda x:torch.tensor([x]),
    lambda x:torch.nn.functional.one_hot(x,10),
    torch.squeeze,
    torch.Tensor.numpy,
])

# Download datasets
dataset_train = datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=feature_transform_pipeline,
    target_transform = label_transform_pipeline,
)

dataset_val = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=feature_transform_pipeline,
    target_transform = label_transform_pipeline,
)
print(f"Number of training samples: {len(dataset_train)}")
print(f"Number of validation samples: {len(dataset_val)}")
print(f"Features shape: {dataset_train[0][0].shape}")
print(f"Classes: {dataset_train.classes}")

# Create dataloaders
batch_size = 128
dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
dataloader_val = DataLoader(dataset_val, batch_size=100000, shuffle=True)


# Visualize data samples
sample_batch_train = next(iter(dataloader_train))
fig, axs = plt.subplots(1, 12, figsize=(12, 4))
for i in range(12):
    sample_img = np.transpose(sample_batch_train[0][i].numpy().reshape(3, 32, 32), (1, 2, 0))  # reconstruct image to (H, W, C) format
    sample_cls = dataset_train.classes[sample_batch_train[1][i].numpy().argmax()]
    axs[i] = plt.subplot(1, 12, i + 1)
    axs[i].set_title(sample_cls)
    axs[i].axis('off')
    axs[i].imshow(sample_img)  # image is already in (H, W, C) format
plt.tight_layout()

## 2. Construct Multi-Layer Perceptron Model
A Multi-Layer Perceptron (MLP) model is featured with multiple layers of features, $\mathbf{X}^{[l]}$ s and $\mathbf{Z}^{[l]}$ s. 
The features of the previous layers, ${[l-1]}$ will be transformed (by a stack of a linear function and a non-linear function) to the features in the next layer, ${[l]} \text{, where } l \in {1, 2, \dots, L}$. 

$$
\begin{equation}
    \mathbf{X}^{[l]} = a(\mathbf{X}^{[l-1]} \cdot \mathbf{W}^{[l]T} + \mathbf{b}^{[l]})
\end{equation}
$$

The input layer can be defined as $\mathbf{X}^{[0]}$, and the output layer can be defined as $\hat{\mathbf{Y}} = \mathbf{X}^{[L]}$, where $L$ is the index of the last layer.

### 2.1. Initialize Parameters
An MLP model is governed by $L$ sets of $\mathbf{W}^{[l]}$ s and $\mathbf{b}^{[l]}$ s.
Assume $\mathbf{X}^{[l-1]}$ has $N_{l-1}$ features and $\mathbf{X}^{[l]}$ has $N_{l}$ features, $\mathbf{W}^{[l]}$ needs to be an array/matrix with shape $(N_l, N_{l-1})$, and $\mathbf{b}^{[l]}$ needs to be an array/vector with shape $(1, N_l)$.


### <font color=#582c83> (5%) Exercise 1: Parameters Initialization </font>
Define a function to initialize weights and biases parameters and save these parameters in a Python **dictionary**. 


In [ ]:
def init_params(in_dims: int, hidden_dims: tuple, out_dims: int) -> dict:
    """
    Initialize the parameters of the MLP model.
    Args:
        in_dims: number of input dimensions.
        hidden_dims: tuple/list of hidden layer dimensions.
        out_dims: number of output dimensions.
    Returns:
        params: dictionary containing the initialized weights and biases.
    """
    layer_dims = (in_dims, *hidden_dims, out_dims)
    params = {}
    for l in range(len(layer_dims) - 1):
        params[f'W{l+1}'] = np.random.normal(loc=0, scale=0.01, size=(layer_dims[l + 1], layer_dims[l]))
        params[f'b{l+1}'] = np.random.normal(loc=0, scale=0.01, size=(1, layer_dims[l + 1]))
    return params

# Example usage
np.random.seed(3321)  # for reproducibility
params_dummy = init_params(in_dims=sample_batch_train[0].numpy().shape[1], hidden_dims=(4, 3), out_dims=sample_batch_train[1].numpy().shape[1])
# print(f"Dummy parameter: {params_dummy}")
for key in params_dummy:
    print(f"{key} shape: {params_dummy[key].shape}")


**Expected Output**:
>
```console
W1 shape: (4, 3072)
b1 shape: (1, 4)
W2 shape: (3, 4)
b2 shape: (1, 3)
W3 shape: (10, 3)
b3 shape: (1, 10)
```

### 2.2 - Activate Features
A non-linear activation function will be applied on $\mathbf{Z}^{[l]}$ to form new features $\mathbf{X}^{[l]}$ as the $l$-th layer in an MLP model.
Commonly use activations are:

- Sigmoid
$$x = \sigma (z) = \frac{1}{1 + e^{-z}}$$

- Hyperbolic Tangent
$$x = \tanh (z) = \frac{e^z - e^{-z}}{e^z + e^{-z}}$$

- Rectified Linear Unit
$$
x = \text{ReLU}(z) = 
    \begin{cases}
        0   & z \leq 0 \\
        z   & z > 0
    \end{cases}
$$

- Leaky ReLU
$$
x = \text{LeakyReLU}(z; a \geq 0) = 
    \begin{cases}
        az   & z \leq 0 \\
        z   & z > 0
    \end{cases}
$$

- Softmax
For a multi-class classification problem, we will want to encode $\mathbf{Z}^{[L]}$ into probability distributions.

$$\hat{\mathbf{y}}_i = \frac{e^{\mathbf{z}^{[L]}_i}}{\sum^C_{i=1} e^{\mathbf{z}^{[L]}_i}}$$

The maxtrix $\mathbf{Z}^{[L]}$ has shape: $(M, C)$, where $M$ is the number of samples, $C$ is the number of classes. 
When applying softmax activation, we only want it to deal with each row of $\mathbf{Z}^{[L]}$, or perform it on the last dimension (axis) of $\mathbf{Z}^{[L]}$.

### <font color=#582c83> (5%) Exercise 2. Activation Functions </font>



In [ ]:
### START CODE HERE ### (≈ 9 lines of code)
def sigmoid(z):
    """ Sigmoid function
    Args:
        z: independent variable, could be an arrary of any shape or a scalar. 
    Returns:
        dependent variable, could be an arrary of any shape or a scalar. 
    """
    return 1 / (1 + np.exp(-z))


def tanh(z):
    """ Hyperbolic Tangent function
    Args:
        z: independent variable, could be an arrary of any shape or a scalar. 
    Returns:
        dependent variable, could be an arrary of any shape or a scalar. 
    """
    return (np.exp(z) - np.exp(-z)) / (np.exp(z) + np.exp(-z))


def relu(z):
    """ ReLU function
    Args:
        z: independent variable, could be an arrary of any shape or a scalar. 
    Returns:
        dependent variable, could be an arrary of any shape or a scalar. 
    """
    return np.maximum(0, z)


def leaky_relu(z, a=0.05):
    """ Leaky ReLU function
    Args:
        z: independent variable, could be an arrary of any shape or a scalar. 
        a: constant, a scalar. 
    Returns:
        dependent variable, could be an arrary of any shape or a scalar. 
    """
    return np.maximum(a * z, z)


def softmax(z):
    """ Softmax function
    Args:
        z: independent variable, 2d array with shape (# samples, # features)
    Returns:
        probabilities: dependent variable, 2d array with shape (# samples, # features)
    """
    probabilities = np.exp(z) / np.sum(np.exp(z), axis=-1, keepdims=True)
    return probabilities

# Sanity check
z_dummy = np.linspace(-2, 2, 10).reshape(2, 5)
print(f"Input = \n{z_dummy}")
print(f"Sigmoid activation = \n{sigmoid(z_dummy)}")
print(f"Hyper Tangent activation = \n{tanh(z_dummy)}")
print(f"ReLU activation = \n{relu(z_dummy)}")
print(f"Leaky ReLU activation = \n{leaky_relu(z_dummy, a=0.1)}")
print(f"Softmax activation = \n{softmax(z_dummy)}")



**Expected Output**:
>
```console
Input = 
[[-2.         -1.55555556 -1.11111111 -0.66666667 -0.22222222]
 [ 0.22222222  0.66666667  1.11111111  1.55555556  2.        ]]
Sigmoid activation = 
[[0.11920292 0.17428532 0.2476638  0.33924363 0.44467194]
 [0.55532806 0.66075637 0.7523362  0.82571468 0.88079708]]
Hyper Tangent activation = 
[[-0.96402758 -0.9146975  -0.8044548  -0.58278295 -0.21863508]
 [ 0.21863508  0.58278295  0.8044548   0.9146975   0.96402758]]
ReLU activation = 
[[0.         0.         0.         0.         0.        ]
 [0.22222222 0.66666667 1.11111111 1.55555556 2.        ]]
Leaky ReLU activation = 
[[-0.2        -0.15555556 -0.11111111 -0.06666667 -0.02222222]
 [ 0.22222222  0.66666667  1.11111111  1.55555556  2.        ]]
Softmax activation = 
[[0.06801606 0.10607944 0.16544399 0.25803034 0.40243017]
 [0.06801606 0.10607944 0.16544399 0.25803034 0.40243017]]
```

### 2.3. Forward Propagation
Refer to (1), The forward pass of an MLP model is looping the linear and activation transforms. 

### <font color=#582c83> (10%) Exercise 3. MLP Forward Pass Construction </font>


In [ ]:
def linear(in_features, weights, biases):
    """ Linear function
    Args:
        in_features: input feature matrix, 2d array with shape (# samples, # input features)
        weights: weight parameter matrix, 2d array with shape (# next layer features , # input features)
        biases: bias parameter vector, 2d array with shape (1, # next layer features)
    Returns:
        linear_output: linear model output feature matrix, 2d array with shape (# samples, # next layer features)
    """
    linear_output = in_features @ weights.T + biases
    return linear_output

def forward(in_features, params, hidden_activation=None, out_activation=None):
    """ Forward function
    Args:
        in_features: feature matrix, 2d array with shape (# samples, # pixels)
        params: a dictionary of weights and biases
            params = {
                'W1': weight matrix of the first layer, 2d array with shape (# hidden features in 1st layer, # input features)
                'b1': bias vector of the first layer, 2d array with shape (1, # hidden features in 1st layer)
                ...
                'Wi': weight matrix of the i-th layer, 2d array with shape (# hidden features in i-th layer, # hidden features in (i-1)-th layer)
                'bi': bias vector of the i-th layer, 2d array with shape (1, # hidden features in i-th layer)
                ...
                'WL': weight matrix of the last layer, 2d array with shape (# hidden features in output layer, # hidden features in (L-1)-th layer)
                'bL': bias vector of the last layer, 2d array with shape (1, # output features)
            }
        hidden_activation: activation function for hidden layers, could be sigmoid, relu, etc. If None, no activation will be applied.
        out_activation: activation function for output layer, could be sigmoid, softmax, etc. If None, no activation will be applied.
    Returns:
        predictions: predicted probabilities, a column vector or 2d array with shape (# samples, # output features)
        cache: linear transformed and activated features
    """

    num_layers = len(params) // 2
    # Input layer
    cache = {'X0': in_features}
    # Hidden layers
    for i in range(num_layers - 1):
        cache[f'Z{i+1}'] = linear(cache[f'X{i}'], params[f'W{i+1}'], params[f'b{i+1}'])
        if hidden_activation=="relu":
            cache[f'X{i+1}'] = relu(cache[f'Z{i+1}'])
        elif hidden_activation=="leaky_relu":
            cache[f'X{i+1}'] = leaky_relu(cache[f'Z{i+1}'])
        elif hidden_activation=="sigmoid":
            cache[f'X{i+1}'] = sigmoid(cache[f'Z{i+1}'])
        elif hidden_activation=="tanh":
            cache[f'X{i+1}'] = tanh(cache[f'Z{i+1}'])
        else:
            cache[f'X{i+1}'] = cache[f'Z{i+1}']
    # Output layer
    cache[f'Z{num_layers}'] = linear(cache[f'X{num_layers-1}'], params[f'W{num_layers}'], params[f'b{num_layers}'])
    if out_activation=="relu":
        predictions = relu(cache[f'Z{num_layers}'])
    elif out_activation=="leaky_relu":
        predictions = leaky_relu(cache[f'Z{num_layers}'])
    elif out_activation=="sigmoid":
        predictions = sigmoid(cache[f'Z{num_layers}'])
    elif out_activation=="tanh":
        predictions = tanh(cache[f'Z{num_layers}'])
    elif out_activation=="softmax":
        predictions = softmax(cache[f'Z{num_layers}'])
    else:
        predictions = cache[f'Z{num_layers}']
    return predictions, cache

# Sanity check
preds_dummy, cache_dummy = forward(sample_batch_train[0].numpy(), params_dummy, "sigmoid", "softmax")
print(f"Features: {list(cache_dummy.keys())[:]}")
print(f"Shape of dummy predictions: {preds_dummy.shape}")
print(f"Dummy prediction sample with sigmoid activations:\n{preds_dummy[0]}")
preds_dummy, cache_dummy = forward(sample_batch_train[0].numpy(), params_dummy, "tanh", "softmax")
print(f"Dummy prediction sample with hyperbolic tangent activations:\n{preds_dummy[0]}")
preds_dummy, cache_dummy = forward(sample_batch_train[0].numpy(), params_dummy, "relu", "softmax")
print(f"Dummy prediction sample with ReLU activations:\n{preds_dummy[0]}")
preds_dummy, cache_dummy = forward(sample_batch_train[0].numpy(), params_dummy, "leaky_relu", "softmax")
print(f"Dummy prediction sample with Leaky ReLU activations:\n{preds_dummy[0]}")


**Reference Output**:
>
```console
Features: ['X0', 'Z1', 'X1', 'Z2', 'X2', 'Z3']
Shape of dummy predictions: (128, 10)
Dummy prediction sample with sigmoid activations:
[0.09528843 0.09125905 0.11383818 0.09590308 0.1024395  0.09614213
 0.08904265 0.09979815 0.10857134 0.1077175 ]
Dummy prediction sample with hyperbolic tangent activations:
[0.10076069 0.09910421 0.09969189 0.10101848 0.09819445 0.10342691
 0.10052092 0.09823987 0.09928544 0.09975714]
Dummy prediction sample with ReLU activations:
[0.09428151 0.09640544 0.10352176 0.09853189 0.10229858 0.09985063
 0.09449275 0.10236338 0.10237048 0.10588358]
Dummy prediction sample with Leaky ReLU activations:
[0.09453645 0.09626262 0.1035647  0.09868714 0.10195887 0.10033994
 0.09460139 0.10198885 0.10230915 0.10575089]
```

## 3. Evaluate Model

### 3.1. Cross Entropy Loss
The cross-entropy (CE) can be used to measure the difference between two probability distributions.
Since the one-hot encoded label and the softmax activated prediction are both probability distributions, the error between them can be calculated using the following equation.  

$$
\mathcal{L}(\hat{\mathbf{Y}}, \mathbf{Y}) = -\frac{1}{M} \sum_{m=1}^M \sum_{c=1}^C {^{(m)} y_c} \log {^{(m)} \hat{y}_c}
$$

### 3.2 Accuracy
The accuracy of model predictions can be computed by the following equation.

$$
\text{Accuracy} = \frac{\text{Correct predictions}}{\text{All predictions}}
$$

A sample can be categorized into the class with highest probability, given the sample's softmax activated prediction.
To find out indices of the highest probability for the samples,

$$
\tilde{\mathbf{Y}} = \argmax(\hat{\mathbf{Y}}, \text{axis}=-1)
$$

### <font color=#582c83> (10%) Exercise 4. Cross Entropy Loss and Classification Accuracy </font>



In [ ]:
### START CODE HERE ### (≈ 1 line of code)
def ce_loss(predictions, labels):
    """ Cross Entropy Loss
    Args:
        predictions: predicted probabilities, 2d array with shape (# samples, # classes)
        labels: true one-hot encoded labels, 2d array with shape (# samples, # classes)
    Returns:
        loss: average cross entropy loss, a scalar value
    """
    sample_wise_losses = (-labels * np.log(predictions + 1e-15)).sum(axis=-1)
    return sample_wise_losses.mean()

def classify(predictions):
    """ (OPTIONAL) Classification from (probablistic) predictions
    Args:
        predictions: predicted probabilities, 2d array with shape (# samples, # classes)
    Returns:
        pred_classes: one-hot encoded predicted classes, 2d array with shape (# samples, # classes)
    """
    one_hot_ids = predictions.argmax(axis=-1)
    pred_classes = np.zeros_like(predictions)
    pred_classes[np.arange(predictions.shape[0]), one_hot_ids] = 1
    return pred_classes

def accuracy(predictions, labels):
    """ Average classification accuracy
    Args:
        predictions: predicted probabilities, 2d array with shape (# samples, # classes)
        labels: true one-hot encoded labels, 2d array with shape (# samples, # classes)
    Returns:
        acc: average classification accuracy, a scalar value
    """
    # predicted_classes = classify(predictions)
    # predictions.argmax(axis=-1)
    acc = np.mean(predictions.argmax(axis=-1) == labels.argmax(axis=-1))
    # acc = np.mean((predicted_classes == labels).all(axis=-1))
    return acc
### END CODE HERE ###

print(f"Dummy model CE loss: {ce_loss(preds_dummy, sample_batch_train[1].numpy())}")
print(f"True label samples: \n{sample_batch_train[1].numpy()[:10]}")
print(f"Dummy classification samples: \n{classify(preds_dummy)[:10]}")
print(f"Dummy model accuracy: {accuracy(preds_dummy, sample_batch_train[1].numpy()) * 100}%")


**Expected Output**:
>
```console
Dummy model CE loss: 2.3025913914571188
True label samples: 
[[0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 1 0 0]]
Dummy classification samples: 
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]
Dummy model accuracy: 9.375%
```

## 4. Back-Propagate Gradeients of Loss
In order to update model parameters (weights and biases), we need compute gradients of the CE loss or partial derivatives of the loss w.r.t. these model parameters.

### 4.1. Derivative of Activation Functions 
The derivatives of activation functions play important roles when computing the gradients of loss.
- Derivative of Sigmoid
$$\sigma'(z) = \sigma(z) (1 - \sigma(z)$$

- Derivative of Hyperbolic Tangent
$$\tanh' (z) = 1 - \tanh^2 (z) = \text{sech}^2 (z)$$

- Derivative of Rectified Linear Unit
$$
\text{ReLU}' (z) = 
    \begin{cases}
        0   & z \leq 0 \\
        1   & z > 0
    \end{cases}
$$

- Derivative of Leaky ReLU
$$
\text{LeakyReLU}' (z; a \geq 0) = 
    \begin{cases}
        a   & z \leq 0 \\
        1   & z > 0
    \end{cases}
$$

### <font color=#582c83> (4%) Exercise 5. Derivatives of Activation Functions </font>


In [ ]:
### START CODE HERE ### (≈ 9 lines of code)
def d_sigmoid(z):
    """ Derivative of sigmoid function
    Args:
        z: independent variable, could be an arrary of any shape or a scalar. 
    Returns:
        dependent variable, could be an arrary of any shape or a scalar. 
    """
    return sigmoid(z) * (1 - sigmoid(z))

def d_tanh(z):
    """ Derivative of hyperbolic tangent function
    Args:
        z: independent variable, could be an arrary of any shape or a scalar. 
    Returns:
        dependent variable, could be an arrary of any shape or a scalar. 
    """
    return 1 - tanh(z) ** 2

def d_relu(z):
    """ Derivative of ReLU function
    Args:
        z: independent variable, could be an arrary of any shape or a scalar. 
    Returns:
        dependent variable, could be an arrary of any shape or a scalar. 
    """
    return (z > 0).astype(float)

def d_leaky_relu(z, a=0.05):
    """ Derivative of Leaky ReLU function
    Args:
        z: independent variable, could be an arrary of any shape or a scalar. 
        a: constant, a scalar. 
    Returns:
        dependent variable, could be an arrary of any shape or a scalar. 
    """
    return np.where(z > 0, 1.0, a)

print(f"Sigmoid derivative = \n{d_sigmoid(z_dummy)}")
print(f"Hyper Tangent derivative = \n{d_tanh(z_dummy)}")
print(f"ReLU derivative = \n{d_relu(z_dummy)}")
print(f"Leaky ReLU derivative = \n{d_leaky_relu(z_dummy, a=0.1)}")


### 4.2. Back Propagation
 
The back-propagation process starts from the last layer and back propagate to the first layer. 

Due to the fact that the last layer is softmax activated, $\frac{\partial \mathcal{L}}{\partial \mathbf{Z}^{[L]}}$ can be computed differently without explicitly solve for derivative of softmax function.

$$
d\mathbf{Z}^{[L]} = \frac{\partial \mathcal{L}}{\partial \mathbf{Z}^{[L]}} = \mathbf{\hat{Y}} - \mathbf{Y}
$$

Then, from last layer $L$ to first layer, we need to repeatedly computing the gradients of loss according to the chain rule. 
The computation of a general layer $[l]$ is as follows.
$$d\mathbf{W}^{[l]} = d\mathbf{Z}^{[l]} \frac{\partial \mathbf{Z}^{[l]}}{\partial{\mathbf{W}^{[l]}}} = d\mathbf{Z}^{[l]T} \cdot \mathbf{X}^{[l-1]}$$
$$d\mathbf{b}^{[l]} = d\mathbf{Z}^{[l]} \frac{\partial \mathbf{Z}^{[l]}}{\partial{\mathbf{b}^{[l]}}} = mean(d\mathbf{Z}^{[l]}, axis=0, keepdims=True)$$
$$d\mathbf{X}^{[l-1]} = d\mathbf{Z}^{[l]} \frac{\partial \mathbf{Z}^{[l]}}{\partial{\mathbf{X}^{[l-1]}}} = d\mathbf{Z}^{[l]} \cdot \mathbf{W}^{[l]}$$
$$d\mathbf{Z}^{[l-1]} = d\mathbf{X}^{[l-1]} * a'(\mathbf{Z}^{[l-1]})$$

Another special case is when $l=1$. 
Since the back-propagation will stop, and $d\mathbf{Z}^{[0]}$ does not exist, there is no need to compute $d\mathbf{X}^{[0]}$ and $d\mathbf{Z}^{[0]}$.

### <font color=#582c83> (18%) Exercise 6. Gradients of Loss Back-Propagation </font>

In [ ]:

def backward(predictions, labels, cache, params, activation="relu"):
    """ Backward function
    Args:
        predictions: predicted probabilities, 2d array with shape (# samples, # output features)
        labels: true one-hot encoded labels, 2d array with shape (# samples, # output features)
        cache: linear transformed and activated features from forward pass
        params: a dictionary of weights and biases
        activation: activation function used in hidden layers during forward pass, could be sigmoid, relu, etc.
    Returns:
        grads: a dictionary of gradients with respect to weights and biases
        grads = {
            'dW1': gradient of the weight matrix of the first layer, 2d array with shape (# hidden features in 1st layer, # input features)
            'db1': gradient of the bias vector of the first layer, 2d array with shape (1, # hidden features in 1st layer)
            ...
            'dWi': gradient of the weight matrix of the i-th layer, 2d array with shape (# hidden features in i-th layer, # hidden features in (i-1)-th layer)
            'dbi': gradient of the bias vector of the i-th layer, 2d array with shape (1, # hidden features in i-th layer)
            ...
            'dWL': gradient of the weight matrix of the last layer, 2d array with shape (# hidden features in output layer, # hidden features in (L-1)-th layer)
            'dbL': gradient of the bias vector of the last layer, 2d array with shape (1, # output features)
        }
    """
### Start CODE HERE ### (≈ 18 lines of code)
    num_layers = len(params) // 2
    grads = {f'dZ{num_layers}': predictions - labels}
    for i in reversed(range(num_layers)):
        grads[f'dW{i+1}'] = grads[f'dZ{i+1}'].T @ cache['X' + str(i)]
        grads[f'db{i+1}'] = np.mean(grads[f'dZ{i+1}'], axis=0, keepdims=True)
        if i==0:
            break  
        grads[f'dX{i}'] = grads[f'dZ{i+1}'] @ params[f'W{i+1}']
        if activation == 'sigmoid':
            grads[f'dZ{i}'] = grads[f'dX{i}'] * d_sigmoid(cache[f'Z{i}'])
        elif activation == 'tanh':
            grads[f'dZ{i}'] = grads[f'dX{i}'] * d_tanh(cache[f'Z{i}'])
        elif activation == 'relu':
            grads[f'dZ{i}'] = grads[f'dX{i}'] * d_relu(cache[f'Z{i}'])
        elif activation == 'leaky_relu':
            grads[f'dZ{i}'] = grads[f'dX{i}'] * d_leaky_relu(cache[f'Z{i}'])
        else:
            grads[f'dZ{i}'] = grads[f'dX{i}']

    return grads
### END CODE HERE ###

grads_dummy = backward(preds_dummy, sample_batch_train[1].numpy(), cache_dummy, params_dummy, "leaky_relu")
for key in grads_dummy:
    print(f"{key} shape: {grads_dummy[key].shape}")
# print(grads_dummy)




**Expected Output**:
>
```console
dZ3 shape: (128, 10)
dW3 shape: (10, 3)
db3 shape: (1, 10)
dX2 shape: (128, 3)
dZ2 shape: (128, 3)
dW2 shape: (3, 4)
db2 shape: (1, 3)
dX1 shape: (128, 4)
dZ1 shape: (128, 4)
dW1 shape: (4, 3072)
db1 shape: (1, 4)
```

## 5. Update Model Parameters
Now, it's time to perform gradient descent algorithm to improve the model.
Due to the size limit of the computer memory, the entire dataset may not be able to be fit into the RAM at once.
Instead, break the dataset down into smaller batches is a more practical solution.
Therefore, the model parameters need to be updated multiple times until all the data samples are used in the training loop.    


### <font color=#582c83> (33%) Exercise 7. Stochastic Gradient Descent (SGD) Optimization </font>

Train your model. Bring both training loss and test loss down. **Note: you may need to spend more time on this task. So, get started as early as possible.**
- Set hyper-parameters: number of iterations(epochs), learning rate, hidden layer sizes.
- Repeat model parameters updating until CE loss converged:
    1. Extract a batch from the dataset. 
    2. Predict batch classes. 
    3. Evaluate model.
    4. Compute gradients.
    5. Update parameters on the loss descending directions.

- > Tip: Choose the activation function wisely. A poorly activated layer may cause longer training duration as well as degraded model performance.

<font color=red> **Bring validation accuracy over 70%** </font>

In [ ]:
### Start CODE HERE ### (≈ 11 lines of code)
# Hyper-parameters
num_iters = 50
learning_rate = 3e-4
hidden_layer_sizes = (32, 32)
# Model parameters
params = init_params(in_dims=sample_batch_train[0].numpy().shape[-1], hidden_dims=hidden_layer_sizes, out_dims=sample_batch_train[1].numpy().shape[-1])
# Reserve metrics storage
losses_train, losses_val = [], []
accuracies_train, accuracies_val = [], []

# LOOP
for ep in range(num_iters):
    ep_loss, ep_acc = [], []
    for bi, batch_train in enumerate(dataloader_train):
        preds_train, cache_train = forward(batch_train[0].numpy(), params, "relu", "softmax")
        batch_loss = ce_loss(preds_train, batch_train[1].numpy())
        batch_acc = accuracy(preds_train, batch_train[1].numpy())
        ep_loss.append(batch_loss)
        ep_acc.append(batch_acc)
        if not (bi+1) % 50:
            print(f"Training epoch {ep+1} batch {bi+1} loss: {batch_loss}, accuracy: {batch_acc}")
        grads = backward(preds_train, batch_train[1].numpy(), cache_train, params, activation="relu")
        for l in range(len(params_dummy) // 2):
            params[f'W{l+1}'] = params[f'W{l+1}'] - learning_rate * grads[f'dW{l+1}']
            params[f'b{l+1}'] = params[f'b{l+1}'] - learning_rate * grads[f'db{l+1}']
    # Validate updated model on entire val dataset
    data_val = next(iter(dataloader_val))
    preds_val, _ = forward(data_val[0].numpy(), params, "relu", "softmax")
    loss_val = ce_loss(preds_val, data_val[1].numpy())
    acc_val = accuracy(preds_val, data_val[1].numpy())
    print(f"Epoch {ep+1} validation accuracy: {acc_val}")    
    # Summarize epoch
    losses_train.append(np.mean(ep_loss))
    accuracies_train.append(np.mean(ep_acc))
    losses_val.append(loss_val)
    accuracies_val.append(acc_val)
### END CODE HERE ###


### 5.1. Exam Training and Validation

In [ ]:
# Plot loss and accuracy curves
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(range(num_iters), losses_train, range(num_iters), losses_val)
plt.legend(['Train', 'Validation'])
plt.title("Loss Decrease")
plt.xlabel("Iteration")
plt.ylabel("BCE Loss")
plt.subplot(1, 2, 2)
plt.plot(range(num_iters), accuracies_train, range(num_iters), accuracies_val)
plt.legend(['Train', 'Validation'])
plt.title("Accuracy Growth")
plt.xlabel("Iteration")
plt.ylabel("Accuracy")
plt.ylim(0, 1)

### 5.2. Confusion Matrix

In [ ]:
# Confusion Matrix 
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

trimmed_classes = [c[:4] for c in dataset_train.classes] 
cm = confusion_matrix(data_val[1].numpy().argmax(axis=-1), preds_val.argmax(axis=-1))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=trimmed_classes)
disp.plot(cmap=plt.cm.PuRd)
plt.title("Confusion Matrix on Validation Data")

In [ ]:
data_val[1].sum(0)

### <font color=#582c83> (15%) Exercise 8. Model Analyses </font>
Based on the validation confusion matrix.
1. Please rank your model's classification accuracies based on the classes. 
2. Please report the top 3 classes your model struggled most. 
3. Take your model's most struggling class as an example, please try to briefly explain why the model was not performing well making correct predictions on this class. 

> Please write your answers below.



## Congrats on finishing this assignment! 